# Анализ базы данных сервиса для чтения книг по подписке

## Описание проекта

**Задачи проекта:** Используя базу данных сервиса для чтения книг по подписке проанализировать следующие данные:
- количество книг, вышедших после 1 января 2000 года
- количество обзоров и средняя оценка по книгам
- издательство, которое выпустило наибольшее число книг (для книг толще 50 страниц)
- автор с самой высокой средней оценкой книг (для книг с 50 и более оценками)
- среднее количество обзоров от пользователей, которые поставили больше 50 оценок

**Данные для анализа:** База данных сервиса для чтения книг по подписке.

## Импорт библиотек и подключение к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': '***', # имя пользователя
             'pwd': '***', # пароль
             'host': '***',
             'port': '****', # порт подключения
             'db': '***'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                 db_config['pwd'],
                                                 db_config['host'],
                                                 db_config['port'],
                                                 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Изучение таблиц

In [2]:
query_books = '''
           SELECT *
           FROM books
        '''

books = pd.io.sql.read_sql(query_books, con = engine)
books.head(5)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица **books** 
содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

In [3]:
query_authors = '''
           SELECT *
           FROM authors
        '''

authors = pd.io.sql.read_sql(query_authors, con = engine)
authors.head(5)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица **authors**
содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

In [4]:
query_publishers = '''
           SELECT *
           FROM publishers
        '''

publishers = pd.io.sql.read_sql(query_publishers, con = engine)
publishers.head(5)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица **publishers**
содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства.

In [5]:
query_ratings = '''
           SELECT *
           FROM ratings
        '''

ratings = pd.io.sql.read_sql(query_ratings, con = engine)
ratings.head(5)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица **ratings**
содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

In [6]:
query_reviews = '''
           SELECT *
           FROM reviews
        '''

reviews = pd.io.sql.read_sql(query_reviews, con = engine)
reviews.head(5)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица **reviews**
содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

### Вывод

База данных сервиса книг по подписке содержит 5 таблиц:
- books — данные о книгах, 
- authors – данные об авторах,
- publishers – данные об издательствах, 
- ratings – данные о пользовательских оценках,
- reviews - данные о пользовательских обзорах.

## Исследовательский анализ данных

<a id="sql01"></a> 
### Количество книг, вышедших после 1 января 2000 года

In [7]:
query_01 = '''
           SELECT *
           FROM books
           WHERE CAST(publication_date AS timestamp) > '2000-01-01'
        '''

books_after_2000 = pd.io.sql.read_sql(query_01, con = engine)
len(books_after_2000)

819

#### Вывод

После 1 января 2000 года было издано 819 книг.

<a id="sql02"></a> 
### Количество обзоров и средняя оценка по книгам

In [8]:
query_02 = '''
           SELECT b.book_id, 
                  b.title,
                  COUNT(DISTINCT rw.review_id) AS n_reviews, 
                  ROUND(AVG(rt.rating), 2) AS avg_rating
           FROM books AS b
           LEFT OUTER JOIN reviews AS rw ON b.book_id = rw.book_id
           LEFT OUTER JOIN ratings AS rt ON b.book_id = rt.book_id
           GROUP BY b.book_id
           ORDER BY avg_rating DESC, n_reviews DESC
        '''

reviews_ratings = pd.io.sql.read_sql(query_02, con = engine)
reviews_ratings

,book_id,title,n_reviews,avg_rating
0,17,A Dirty Job (Grim Reaper #1),4,5.00
1,553,School's Out—Forever (Maximum Ride #2),3,5.00
2,444,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,642,The Big Bad Wolf (Alex Cross #9),2,5.00
4,967,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,202,Drowning Ruth,3,2.00
997,316,His Excellency: George Washington,2,2.00
998,371,Junky,2,2.00


<a id="sql03"></a> 
### Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [9]:
query_03 = '''
           SELECT p.publisher, COUNT(DISTINCT b.book_id) AS n_books
           FROM publishers AS p
           LEFT OUTER JOIN books AS b ON p.publisher_id = b.publisher_id
           WHERE b.num_pages > 50
           GROUP BY p.publisher
           ORDER BY n_books DESC
           LIMIT 1
        '''

top_publisher = pd.io.sql.read_sql(query_03, con = engine)
top_publisher

,publisher,n_books
0,Penguin Books,42


#### Вывод

Издательство, которое выпустило наибольшее число книг толще 50 страниц — Penguin Books — 42 книги.

<a id="sql04"></a> 
### Автор с самой высокой средней оценкой книг (для книг с 50 и более оценками)

In [10]:
query_04 = '''
              SELECT books_rating.author_id, a.author, AVG(books_rating.rating) AS avg_rating
              FROM (           
               SELECT b.author_id, rt.book_id, rt.rating
               FROM ratings AS rt
               LEFT OUTER JOIN books AS b ON rt.book_id = b.book_id
               WHERE rt.book_id IN (SELECT rt.book_id
                                   FROM ratings AS rt
                                   GROUP BY rt.book_id
                                   HAVING COUNT(rt.rating_id) >= 50) 
                    ) AS books_rating
               INNER JOIN authors AS a ON a.author_id = books_rating.author_id
               GROUP BY books_rating.author_id, a.author
               ORDER BY avg_rating DESC
               LIMIT 1
                '''
         
         
q04 = pd.io.sql.read_sql(query_04, con = engine)
q04

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


#### Вывод

Автор с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками) — J.K. Rowling/Mary GrandPré.

<a id="sql05"></a> 
### Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [11]:
query_05 = '''
        SELECT AVG(n_reviews.n_reviews) AS avg_reviews_qty
        FROM (SELECT rw.username, COUNT(rw.review_id) AS n_reviews
              FROM reviews AS rw
              WHERE rw.username IN (SELECT rt.username
                                    FROM ratings AS rt
                                    GROUP BY rt.username
                                    HAVING COUNT(rt.rating_id) > 50)
              GROUP BY rw.username) AS n_reviews
            '''
         
         
avg_reviews = pd.io.sql.read_sql(query_05, con = engine)
avg_reviews

,avg_reviews_qty
0,24.333333


#### Вывод

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок — 24.3.